<a href="https://colab.research.google.com/github/TheQuanEnthusiast/MachineLearning/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Let's import movie_reviews data from nltk
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [7]:
#See the volume of the data
len(movie_reviews.words())

1583820

In [81]:
#As we see from the previous cell, it will be somehow difficult and inappropriate for us to handle this volume of data. That's why it is recommended to use subset of this data.
#We will use movie_reviews data with volume of 50k from Kaggle.
import numpy as np
import pandas as pd
data = pd.read_csv('/content/IMDB Dataset.csv')

In [82]:
data.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [83]:
data.shape

(50000, 2)

In [84]:
#As we see, our data is in categorical form. It's always recommended to pass in training data on to machine learning algorithm in numerical form.
#Here we will use bag-of-words model to covert text data into numerical form. The idea behind bag-of-words model is as follow:
#1.We create a vocabulary of a unique token, subset texts fro the entire documents
#2.We create a feature vector from each document that contains the numbers of how much each word occurs in a document.
#To construct a bag-of-words model corresponding to the counts of each word we will use CountVectorizer class implemeted in the scikit-learn.
#CountVectorizer takes an array of words and creates a bag-of-words model.
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
c = CountVectorizer()
bag_of_words = c.fit_transform(data)

# <h>Data Cleaning</h>

In [85]:
#Let's look at our data and see how important is data cleaning. As we see from the result of following code, data contains HTML markup, punctuation and some emoticons.
#HTML markup doesn't contain certain discriminatory information. Punctuation may be contains but we will get rid of it for simplicity. And we will keep emoticons
#as they provide great deal in sentiment analysis.
data.loc[1, 'review']

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [86]:
#Let's do the previous task with the following lines of code
#We use Python's regular expression library
import re
def clean(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
  return text

In [87]:
#Let's check if our code done well
clean(data.loc[1, 'review'][-100:])

'particularly of their flat with halliwell s murals decorating every surface are terribly well done '

In [88]:
data['review'] = data['review'].apply(clean)

**Processing documents into tokens**

In [127]:
#After getting ready our dataset for analysis we need to split text into individual parts. This is also known as tokenization.
def tokenizer(text):
  text = ' '.join(text)
  return text.split()

In [129]:

#Another technique for the previous is word-stemming technique
#This technique will reduce the words into their root form
from nltk.stem.porter import PorterStemmer
p = PorterStemmer()
def tokenize_words(text):
  text = ' '.join(text)
  return [p.stem(word) for word in text.split()]

In [130]:
data['review'] = data['review'].apply(tokenize_words)

In [131]:
#Another step we will take is stop-words removal. Stop words don't carry any useful information. And we will use 127 stop-wrds from nltk library
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## <h>Training a logistic regression for document classificiation</h>

In [132]:
#Next we will train logistic regression to calssify reviews as positive and negative using bag-of-words model
#We will divide dataset into test and training dataset
X_train = data.loc[:25000, 'review'].values
y_train = data.loc[:25000, 'sentiment'].values
X_test = data.loc[25000:, 'review'].values
y_test = data.loc[25000:, 'sentiment'].values

In [ ]:
#Next we will use GridSearchCV object to find best set of parameters for our logistic regression model.
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(lowercase=None, strip_accents=None, preprocessor=None)
parameters_grid = [{'vect__ngram_range': [(1, 1)],
                    'vect__stop_words': [stop, None],
                    'vect__tokenizer': [tokenizer, tokenize_words],
                    'clf__penalty': ['l1', 'l2'],
                    'clf__C': [1.0, 10.0, 100.0]},
                   {'vect__ngram_range': [(1, 1)],
                    'vect__stop_words': [stop, None],
                    'vect__tokenizer': [tokenizer, tokenize_words],
                    'vect__use_idf': [False],
                    'vect__norm': [None],
                    'clf__penalty': ['l1', 'l2'],
                    'clf__C': [1.0, 10.0, 100.0]}
                   ]
                   
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
grid_search_cv_lr_tfidf = GridSearchCV(lr_tfidf, parameters_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)
grid_search_cv_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

...

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', '

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', '

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


In [115]:
#As we have finished our grid search, let's see best parameters
print('Best parameter set: %s ' % grid_search_cv_lr_tfidf.best_params_)

In [ ]:
print('CV accuracy: %.3f ' % grid_search_cv_lr_tfidf.best_score_)

In [ ]:
clf = grid_search_cv_lr_tfidf.best_estimator_
print('Test accuracy: %.3f ' % clf.score(X_test, y_test))